In [1]:
import pandas as pd

from leagueGetData import getLeagueData
from leagueDataCleaner import cleanLeagueData

The players' information.

In [2]:
targets = pd.DataFrame(columns=['pseudo', 'tagline', 'region', 'birthDate'], data=[['Katiounette', 'KITTY', 'europe', '01/10/2001'], ['OhSS', '6121', 'europe', '01/01/2000']])
targets

,pseudo,tagline,region,birthDate
0,Katiounette,KITTY,europe,01/10/2001
1,OhSS,6121,europe,01/01/2000


First, I retrieve game data for one or more player(s). The match list and the data for each match is saved for each player in data/matchLists and data/parsedData. (The path can be modified)

Here, I get data for the first time and it's saved on the computer.

In [3]:
targetID = getLeagueData(targets)
targetID

Saved matchs list for Katiounette#KITTY
Error 404 for match EUW1_6343678419 for Katiounette#KITTY
Error 404 for match EUW1_6343624912 for Katiounette#KITTY
Error 404 for match EUW1_6343264517 for Katiounette#KITTY
Error 404 for match EUW1_6343219631 for Katiounette#KITTY
Saved parsed data for Katiounette#KITTY
Saved matchs list for OhSS#6121
Saved parsed data for OhSS#6121


,puuid,pseudo,tagline,region
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,Katiounette,KITTY,europe
1,XVHOCosPT0oX0q7PpWuFaj8Ll2bP9O7fscbCLRVx0tsNxK...,OhSS,6121,europe


Here, the data is retrieved from the computer.

In [4]:
targetID = getLeagueData(targets)
targetID

Got parsed data from os for Katiounette#KITTY
Got parsed data from os for OhSS#6121


,puuid,pseudo,tagline,region
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,Katiounette,KITTY,europe
1,XVHOCosPT0oX0q7PpWuFaj8Ll2bP9O7fscbCLRVx0tsNxK...,OhSS,6121,europe


I join my two DataFrames.

In [5]:
players = pd.merge(targets, targetID, on=['pseudo', 'tagline', 'region'])
players

,pseudo,tagline,region,birthDate,puuid
0,Katiounette,KITTY,europe,01/10/2001,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...
1,OhSS,6121,europe,01/01/2000,XVHOCosPT0oX0q7PpWuFaj8Ll2bP9O7fscbCLRVx0tsNxK...


Then, I clean the data to get a DataFrame with one line per player.

In [6]:
cleanData = cleanLeagueData(players)
cleanData

,pseudo,tagline,region,birthDate,puuid,dayMostFreq,hourMostFreq,monday,tuesday,wednesday,...,primaryVar32,primaryVar13,primaryVar23,primaryVar33,secondaryVar10,secondaryVar20,secondaryVar30,secondaryVar11,secondaryVar21,secondaryVar31
0,Katiounette,KITTY,europe,01/10/2001,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,6.0,2.0,0.168675,0.120482,0.120482,...,0.000000,303.409639,0.168675,0.0,105.180723,5.445783,1.566265,72.674699,29.144578,0.000000
1,OhSS,6121,europe,01/01/2000,XVHOCosPT0oX0q7PpWuFaj8Ll2bP9O7fscbCLRVx0tsNxK...,0.0,2.0,0.187779,0.138599,0.152012,...,7.393443,323.171386,3.154993,0.0,1278.672131,112.065574,1.989568,608.919523,73.602086,0.448584


Test classification